# Nature Conservancy Fish Classification - Bounding Box End-to-End Model

### Imports & environment

In [1]:
import os
from PIL import Image

from keras.callbacks import ModelCheckpoint
from keras.layers import GlobalAveragePooling2D, Activation, Input, Flatten
from keras.optimizers import Adam
from keras.models import Sequential
from keras.preprocessing.image import load_img, img_to_array

from vgg16bn import Vgg16BN as VggConv

from utils import * 
from models import Vgg16BN, Inception, Resnet50
from glob import iglob

ROOT_DIR = os.getcwd()
DATA_HOME_DIR = ROOT_DIR + '/data'
%matplotlib inline

Using Theano backend.
Using gpu device 0: GeForce GTX 980M (CNMeM is enabled with initial size: 90.0% of memory, cuDNN 5105)


### Config & Hyperparameters

In [10]:
# paths
data_path = DATA_HOME_DIR + '/cropped/' 
split_train_path = data_path + 'train/'
valid_path = data_path + 'valid/'
test_path = DATA_HOME_DIR + '/test/'
saved_model_path = 'models/sliding_window/'
pretrained_model_path = "models/bb_end_to_end/0.31-loss_8epoch_224x224_aug_0.001lr_run0_vggbn.h5"
submission_path = 'submissions/sliding_window/'
fish_detector_path = 'models/fish_detector_480x270/0.03-loss_2epoch_480x270_0.3-dropout_0.001-lr_vggbn.h5'

# data
batch_size = 16
im_size = (224, 224)  # (ht, wt); only 299x299 for inception
nb_split_train_samples = 2847
nb_valid_samples = 450
nb_test_samples = 1000
classes = ["ALB", "BET", "DOL", "LAG", "OTHER", "SHARK", "YFT"]
nb_classes = len(classes)

# model
nb_runs = 1
nb_epoch = 12
nb_aug = 1
dropout = 0.5
lr=0.001           
clip = 0.01
archs = ["vggbn"]
WINDOW_SIZE = 224

models = {
    "vggbn": Vgg16BN(size=im_size, n_classes=nb_classes, lr=lr,
                           batch_size=batch_size, dropout=dropout),
    "inception": Inception(size=(299, 299), n_classes=nb_classes,
                           lr=0.001, batch_size=batch_size),
    "resnet": Resnet50(size=im_size, n_classes=nb_classes, lr=lr,
                    batch_size=batch_size, dropout=dropout)
} 

### Build & Train Classifier

In [3]:
def train(parent_model, model_str):
    parent_model.build()    
    model_fn = saved_model_path + '{val_loss:.2f}-loss_{epoch}epoch_' + model_str
    ckpt = ModelCheckpoint(filepath=model_fn, monitor='val_loss',
                           save_best_only=True, save_weights_only=True)
    
    parent_model.fit_val(split_train_path, valid_path, nb_trn_samples=nb_split_train_samples, 
                         nb_val_samples=nb_valid_samples, nb_epoch=nb_epoch, callbacks=[ckpt], aug=nb_aug)

    model_path = max(iglob(saved_model_path + '*.h5'), key=os.path.getctime)
    return model_path

In [4]:
def train_all():    
    model_paths = {
        "vggbn": [],
        "inception": [],
        'resnet': [],
    }
    
    for run in range(nb_runs):
        print("Starting Training Run {0} of {1}...\n".format(run+1, nb_runs))
        aug_str = "aug" if nb_aug else "no-aug"
        
        for arch in archs:
            print("Training {} model...\n".format(arch))
            model = models[arch]
            model_str = "{0}x{1}_{2}_{3}lr_run{4}_{5}.h5".format(model.size[0], model.size[1], aug_str,
                                                                 model.lr, run, arch)
            model_path = train(model, model_str)
            model_paths[arch].append(model_path)
        
    print("Done.") 
    return model_paths
        
# model_paths = train_all()

### Generate Predictions

In [20]:
def generate_preds(model, step=40, win_size=WINDOW_SIZE):
    fish_detector = Vgg16BN(size=(270, 480), n_classes=2, lr=0.001,
                            batch_size=batch_size, dropout=dropout)
    fish_detector.build()
    fish_detector.model.load_weights(fish_detector_path)

    nofish_prob, filenames = fish_detector.test(test_path, nb_test_samples, aug=nb_aug)
    nofish_prob = nofish_prob[:, 1]
    
    predictions = []
    
    for ix, fn in enumerate(filenames):
        
        if ix % 100 == 0:
            print("Processing {0} of {1}".format(ix, len(filenames)))
        
        im_id = fn.split("/")[-1]
        im = img_to_array(load_img(test_path + fn))
    
        # Nofish
        if nofish_prob[ix] > 0.5:
            alloc = (1. - nofish_prob[ix] / 7.)
            pred = np.array([alloc, alloc, alloc, alloc, nofish_prob[ix], alloc, alloc, alloc])
            predictions.append(pred)
            
        # Fish
        else:
            fish_preds = np.zeros(nb_classes)
            box_ix = 0
            
            for top in range(0, im.shape[1] - win_size + 1, step):
                for left in range(0, im.shape[2] - win_size + 1, step):
                    # compute the (top, left, bottom, right) of the bounding box
                    box = (top, left, top + win_size, left + win_size)

                    # crop the original image
                    cropped_img = im[:, box[0]:box[2], box[1]:box[3]].reshape(1, 3, im_size[0], im_size[1])

                    fish_preds += model.predict(cropped_img)[0]
                    box_ix += 1
                    
            fish_preds /= box_ix
            
            fish_preds = np.expand_dims(fish_preds, axis=0)
                    
            weight = -1. * (nofish_prob[ix] - 1.)
            fish_preds = weight * fish_preds
            pred = np.insert(fish_preds, 4, nofish_prob[ix], axis=1)
            predictions.append(pred)
    
    return np.array(predictions), filenames

In [21]:
def test(model_paths):   
                 
    print("----Predicting on {} model...".format(archs[0]))
    parent = models["vggbn"]
    model = parent.build()
    model.load_weights(model_paths[archs[0]][0])
    pred, filenames = generate_preds(model)
    
    predictions = []

    # weird, hacky reshaping
    for p in pred:
        if p.shape != (1, 8):
            p = p.reshape(1, 8)
        predictions.append(p)
        
    predictions = np.concatenate([p for p in predictions])
    return predictions, filenames

predictions, filenames = test(model_paths)

----Predicting on vggbn model...


/home/robert/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/home/robert/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/home/robert/anaconda3/lib/python3.5/site-packages/keras/backend/theano_backend.py:1500: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


Found 1000 images belonging to 1 classes.
Processing 0 of 1000
Processing 100 of 1000
Processing 200 of 1000
Processing 300 of 1000
Processing 400 of 1000
Processing 500 of 1000
Processing 600 of 1000
Processing 700 of 1000
Processing 800 of 1000
Processing 900 of 1000


### Write Predictions to File

In [23]:
def write_submission(predictions, filenames):
    preds = np.clip(predictions, clip, 1-clip)
    sub_fn = submission_path + '{0}epoch_{1}aug_{2}clip_{3}runs'.format(nb_epoch, nb_aug, clip, nb_runs)
    
    for arch in archs:
        sub_fn += "_{}".format(arch)

    with open(sub_fn + '.csv', 'w') as f:
        print("Writing Predictions to CSV...")
        f.write('image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT\n')
        for i, image_name in enumerate(filenames):
            pred = ['%.6f' % p for p in preds[i, :]]
            f.write('%s,%s\n' % (os.path.basename(image_name), ','.join(pred)))
        print("Done.")

write_submission(predictions, filenames)

Writing Predictions to CSV...
Done.
